# TruBot testing

This notebook tests a conversation bot with vector-store context of TruEra website. 

In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

sys.path.append(str(Path().cwd().parent.resolve()))

import pprint
pp = pprint.PrettyPrinter()

# Uncomment to get more debugging printouts:
"""
import logging

root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)
"""

"\nimport logging\n\nroot = logging.getLogger()\nroot.setLevel(logging.DEBUG)\n\nhandler = logging.StreamHandler(sys.stdout)\nhandler.setLevel(logging.DEBUG)\nformatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')\nhandler.setFormatter(formatter)\nroot.addHandler(handler)\n"

In [2]:
# help(openai.Completion.create)

In [3]:
import openai

def wrap(func):
    def wrapped(*args, **kwargs):
        print("wrapping {func.__name__}:")
        pp.pprint(args)
        pp.pprint(kwargs)
        rets = func(*args, **kwargs)
        print("rets=")
        pp.pprint(rets)
        return rets
    
    return wrapped

for attr in dir(openai):
    obj = getattr(openai, attr)
    if hasattr(obj, "create"):
        print(f"wrapping {obj}")
        request = getattr(obj, "create")
        setattr(obj, "create", wrap(request))



wrapping <class 'openai.api_resources.chat_completion.ChatCompletion'>
wrapping <class 'openai.api_resources.completion.Completion'>
wrapping <class 'openai.api_resources.customer.Customer'>
wrapping <class 'openai.api_resources.deployment.Deployment'>
wrapping <class 'openai.api_resources.edit.Edit'>
wrapping <class 'openai.api_resources.embedding.Embedding'>
wrapping <class 'openai.api_resources.file.File'>
wrapping <class 'openai.api_resources.fine_tune.FineTune'>
wrapping <class 'openai.api_resources.image.Image'>
wrapping <class 'openai.api_resources.moderation.Moderation'>


In [4]:
dir(openai)

['APIError',
 'Audio',
 'Callable',
 'ChatCompletion',
 'Completion',
 'ContextVar',
 'Customer',
 'Deployment',
 'Edit',
 'Embedding',
 'Engine',
 'ErrorObject',
 'File',
 'FineTune',
 'Image',
 'InvalidRequestError',
 'Model',
 'Moderation',
 'OpenAIError',
 'Optional',
 'TYPE_CHECKING',
 'Union',
 'VERSION',
 '__all__',
 '__annotations__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'aiosession',
 'api_base',
 'api_key',
 'api_key_path',
 'api_requestor',
 'api_resources',
 'api_type',
 'api_version',
 'app_info',
 'ca_bundle_path',
 'datalib',
 'debug',
 'enable_telemetry',
 'error',
 'log',
 'openai_object',
 'openai_response',
 'organization',
 'os',
 'proxy',
 'requestssession',
 'sys',
 'util',
 'verify_ssl_certs',
 'version']

In [5]:
from trulens_eval.keys import *
from examples.trubot import get_or_make_app, get_answer
from trulens_eval.util import TP
from trulens_eval import Tru

from pprint import PrettyPrinter
pp = PrettyPrinter()

# Tru().reset_database()

KEY SET: OPENAI_API_KEY
KEY SET: PINECONE_API_KEY
KEY SET: PINECONE_ENV
KEY SET: HUGGINGFACE_API_KEY
KEY SET: SLACK_TOKEN
KEY SET: SLACK_SIGNING_SECRET
KEY SET: COHERE_API_KEY


huggingface api: 0requests [00:00, ?requests/s]

openai api: 0requests [00:00, ?requests/s]

In [6]:
app = get_or_make_app(cid=None)

'Starting a new conversation with 0/default.'
✅ app 0/default -> default.sqlite
✅ feedback def. feedback_definition_hash_1e5a0026a052a983a0e7ac5b56855361 -> default.sqlite
✅ feedback def. feedback_definition_hash_c87d35dd01ea669b5beac5e23d141824 -> default.sqlite
✅ feedback def. feedback_definition_hash_280934924ae61c16c4c17c9707db56f4 -> default.sqlite


In [7]:
# List the app components that got instrumented along with their categories.

list(app.instrumented())

[(JSONPath().app, <trulens_eval.utils.langchain.Other at 0x12f547f70>),
 (JSONPath().app.memory, <trulens_eval.utils.langchain.Other at 0x12ff86280>),
 (JSONPath().app.memory.llm,
  <trulens_eval.utils.langchain.LLM at 0x12ff866a0>),
 (JSONPath().app.memory.prompt,
  <trulens_eval.utils.langchain.Prompt at 0x12ff86670>),
 (JSONPath().app.memory.chat_memory,
  <trulens_eval.utils.langchain.Other at 0x12ff86640>),
 (JSONPath().app.combine_docs_chain,
  <trulens_eval.utils.langchain.Other at 0x12ff911c0>),
 (JSONPath().app.combine_docs_chain.llm_chain,
  <trulens_eval.utils.langchain.Other at 0x12ff91160>),
 (JSONPath().app.combine_docs_chain.llm_chain.prompt,
  <trulens_eval.utils.langchain.Prompt at 0x12ff918e0>),
 (JSONPath().app.combine_docs_chain.llm_chain.llm,
  <trulens_eval.utils.langchain.LLM at 0x12ff91d60>),
 (JSONPath().app.combine_docs_chain.document_prompt,
  <trulens_eval.utils.langchain.Prompt at 0x12f547fd0>),
 (JSONPath().app.question_generator,
  <trulens_eval.utils.lan

In [8]:
# proc = Tru().start_dashboard(force=True, _dev=Path.cwd().parent)
# thread = Tru().start_evaluator(restart=True)

In [9]:
selectors = [0,1,3,4]
messages = ["Who is Shayak?", "Wer ist Shayak?", "Kim jest Shayak?", "¿Quién es Shayak?", "Was ist QII?", "Co jest QII?"]

selectors = selectors[2:3]
messages = messages[2:3]

def test_bot(selector, question):
    print(selector, question)
    app = get_or_make_app(cid=question + str(selector), selector=selector)
    answer = get_answer(app=app, question=question)
    return answer

results = []

for s in selectors:
    for m in messages:
        results.append(TP().promise(test_bot, selector=s, question=m))

TP().finish()

3 Kim jest Shayak?
'Starting a new conversation with 3/filtered_context.'
✅ app 3/filtered_context -> default.sqlite
✅ feedback def. feedback_definition_hash_1e5a0026a052a983a0e7ac5b56855361 -> default.sqlite
✅ feedback def. feedback_definition_hash_c87d35dd01ea669b5beac5e23d141824 -> default.sqlite
✅ feedback def. feedback_definition_hash_280934924ae61c16c4c17c9707db56f4 -> default.sqlite
wrapping {func.__name__}:
()
{'engine': 'text-embedding-ada-002',
 'headers': None,
 'input': ['Kim jest Shayak?'],
 'request_timeout': None}
rets=
{'data': [{'embedding': [0.0027410038746893406,
                         -0.01379491575062275,
                         0.0017325368244200945,
                         -0.019992820918560028,
                         -0.03629827871918678,
                         0.01608317159116268,
                         -0.0105913570150733,
                         -0.004628815222531557,
                         -0.01015331968665123,
                         -0.001732

0